# DataTable_RDF_development

This Notebook steps through the development of a method to convert a UKDS DataTable .tab file to a RDF file.

## Initial setup

### Import packages

In [3]:
import os, ukds
import pandas as pd

### Set filepaths

This sets a filepath to an example data dictionary on a local file system, in this case the 'uktus15_household_ukda_data_dictionary.rtf' file.

In [4]:
base_dir=os.path.join(*[os.pardir]*4,r'_Data\United_Kingdom_Time_Use_Survey_2014-2015\UKDA-8128-tab')
dt_fp=os.path.join(base_dir,r'tab\uktus15_household.tab')

### Create DataTable

A ukds.DataTable instance is created and the .tab file is read into it.

In [3]:
dt=ukds.DataTable(dt_fp)

### Print first five rows

In [5]:
dt.tab.head()

,serial,strata,psu,HhOut,hh_wt,IMonth,IYear,DM014,DM016,DM510,...,Relate10_P1,Relate10_P2,Relate10_P3,Relate10_P4,Relate10_P5,Relate10_P6,Relate10_P7,Relate10_P8,Relate10_P9,Relate10_P10
0,11010903,-2,-2,598,,9,2014,0,0,0,...,-2,-2,,,,,,,,
1,11010904,-2,-2,598,,9,2014,0,0,0,...,-2,-2,,,,,,,,
2,11010906,-2,-2,598,,10,2014,0,0,0,...,-2,-2,-2,,,,,,,
3,11010907,-2,-2,598,,9,2014,1,1,0,...,-2,-2,-2,,,,,,,
4,11010908,-2,-2,598,,9,2014,0,0,0,...,-2,,,,,,,,,


## Discussion

### Aim

The aim here is to convert the data in the .tab file into RDF data. This RDF data could then be combined with RDF data from the associated data dictionary and other table data to enable queries over multiple tables and/or multiple UKDS datasets.

The RDFlib Python library is used to make the conversion.

The aim can be refined to: **create a method for the DataTable class which has an argument of a RDFlib Graph instance and returns the same Graph instance populated with the DataTable .tab data.**

### Sample call

Sample code could look like:

```python
import rdflib
g=rdflib.Graph()
g=dt.to_rdf(g) # dt is a DataTable instance
```

### Format of RDF file

The sample code above would output rdf data in the form of a RDFlib Graph instance. Assuming the intial graph was empty, what would the returned RDF data look like?

The proposal is the RDF file would look as below. This shows the data in turtle (.ttl) format for the first five variables in the first row of the data table:

```turtle
@prefix o8128: <http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ukds: <http://purl.org/berg/ontology/UKDS/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

_:0 o8128:serial "11010903" ;
    o8128:strata "-2" ;
    o8128:psu "-2" ;
    o8128:HhOut "598" ; 
    o8128:hh_wt "" ;
```

Here a blank node is used to represent each row in the table. Each variable is associated to the blank node using predicates of the form *ontology_base_uri/variable_name*, such as `o8128:serial`. The value of each triple is the string value imported from the .tab file.

The meaning behind each of these variables, such as their label, missing data values and value labels are defined in a separate RDF file created using the DataDictionary associated with DataTable. This RDF ontology of the table variables can be used to further process the RDF table data (such as to replace the "-2" value of the `o8128:strata` predicate above with the value label value - in this case "Schedule not applicable").

To query multiple tables, it could be useful to create a resource that can be queried directly. For example in the RDF above a resource could be created to represent the household (such as `prefix_uri:11010903`) and this resource used in place of the blank node above. This could be done in multiple tables to join the table datasets together. This is left as a further data processing task if needed.

### Namespaces

#### `o8128: <http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/>`

See the DataDictionary_RDF_development notebook for a description of the choice of this namespace uri to represent the ukds table variables.

## Developing the method

### Imports

In [6]:
import rdflib
from rdflib.namespace import RDF

### Set up input variables

In [8]:
g=rdflib.Graph() # an empty graph
dd_prefix='o8128' # the prefix for the Data Dictionary uri
dd_uri=r'http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/' # the Data Dictionary uri

### Set up namespaces and bind them to the graph

In [9]:
dd_namespace=rdflib.Namespace(dd_uri)
g.bind(dd_prefix,dd_namespace)

### Function to add .tab data to the rdflib graph

In [18]:
def add_row_data(graph,data_dict):
    """Adds the data to the graph
    
    Arguments:
        - graph (rdflib.Graph):
        - data_dict (dict): a dictionary of the row data {variable:value}
    
    """
    
    a=rdflib.BNode()
    
    for k,v in data_dict.items():
        graph.add((a,dd_namespace[k],rdflib.Literal(v)))
        
    return graph

In [19]:
g.update("DELETE WHERE { ?s ?p ?o }")
g=add_row_data(g,dt.tab.loc[0].to_dict())
print(g.serialize(format='ttl').decode())

@prefix o8128: <http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] o8128:Accom "1" ;
    o8128:BenOth "2" ;
    o8128:CCPersNo "-1" ;
    o8128:Cable "2" ;
    o8128:CableNum "-1" ;
    o8128:Comp "1" ;
    o8128:CompNum "1" ;
    o8128:DM014 "0" ;
    o8128:DM016 "0" ;
    o8128:DM1115 "0" ;
    o8128:DM1619 "0" ;
    o8128:DM510 "0" ;
    o8128:DMSex_P1 "1" ;
    o8128:DMSex_P10 "" ;
    o8128:DMSex_P2 "2" ;
    o8128:DMSex_P3 "" ;
    o8128:DMSex_P4 "" ;
    o8128:DMSex_P5 "" ;
    o8128:DMSex_P6 "" ;
    o8128:DMSex_P7 "" ;
    o8128:DMSex_P8 "" ;
    o8128:DMSex_P9 "" ;
    o8128:DVAge_P1 "80" ;
    o8128:DVAge_P10 "" ;
    o8128:DVAge_P2 "71" ;
    o8128:DVAge_P3 "" ;
    o8128:DVAge_P4 "" ;
    o8128:DVAge_P5 "" ;
    o8128:DVAge_P6 "" ;
    o8128:DVA

## Putting it all together

### Final method for the DataDictionary class

In [30]:
import rdflib
from rdflib.namespace import RDF

def to_rdf(self,graph,prefix,uri):
    """Places the DataTable data in an rdflib Graph.
    
    Arguments:
        - graph (rdflib.Graph): a graph to place the data in
        - prefix (str): a prefix for the Data Dictionary ontology (used to describe the variables)
        - uri (str): a uri for the Data Dictionary ontology (used to describe the variables)
    
    Returns:
        - (rdflib.Graph): the input graph with the DataTable data inserted into it.
        
    """
    
    dd_namespace=rdflib.Namespace(dd_uri)
    graph.bind(dd_prefix,dd_namespace)
    
    for index,row in self.tab.iterrows():
        
        data_dict=row.to_dict()
        
        a=rdflib.BNode()
    
        for k,v in data_dict.items():
            graph.add((a,dd_namespace[k],rdflib.Literal(v)))
            
    return graph

In [31]:
kls=ukds.DataTable
kls.to_rdf=to_rdf
dt=kls()
dt.read_tab(dt_fp)
g=rdflib.Graph()
g=dt.to_rdf(graph=g,prefix='o8128',uri='http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/')
len(g)

1585555

In [32]:
g.serialize('tab.ttl',format='ttl')

## to_ttl

In [28]:
l=list(range(10))
a=iter(l)
while True:
    try:
        print(next(a))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [49]:
def to_ttl(self,filename,prefix,uri):
    """Places the DataTable data in an rdflib Graph.
    
    Creates multiple files if size > 30 MB
    
    Arguments:
        - filename (str): the name of the output .ttl file - NO EXTENSION
        - prefix (str): a prefix for the Data Dictionary ontology (used to describe the variables)
        - uri (str): a uri for the Data Dictionary ontology (used to describe the variables)
    
    Returns:
        - (rdflib.Graph): the input graph with the DataTable data inserted into it.
        
    """
    file_index=0
    i=self.tab.iterrows()
    
    while True:
        
        with open(filename+'_'+str(file_index)+'.ttl','w',encoding="UTF-8") as file:
            file.write('@prefix %s: <%s> .\n' % (prefix,uri))
            file.write('\n')
    
            while True: 

                try:
                    index,row = next(i)
                except StopIteration:
                    return
                
                data_dict=row.to_dict()

                l=[]
                for k,v in data_dict.items():
                    l.append('%s:%s "%s"' % (prefix,k,v))

                file.write('[] %s . \n\n' % ' ;\n\t'.join(l))
    
                if index%500==0:
                    filesize_mb=os.path.getsize(filename+'_'+str(file_index)+'.ttl')/(1024*1024.0)
                    if filesize_mb>40:
                        file_index+=1
                        break

In [50]:
kls=ukds.DataTable
kls.to_ttl=to_ttl
dt=kls()
dt.read_tab(dt_fp)
dt.to_ttl('tab',prefix='o8128',uri='http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/')
with open('tab_0.ttl','r') as file:
    print(file.read()[:10000])

@prefix o8128: <http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/> .

[] o8128:serial "11010903" ;
	o8128:strata "-2" ;
	o8128:psu "-2" ;
	o8128:HhOut "598" ;
	o8128:hh_wt "" ;
	o8128:IMonth "9" ;
	o8128:IYear "2014" ;
	o8128:DM014 "0" ;
	o8128:DM016 "0" ;
	o8128:DM510 "0" ;
	o8128:DM1115 "0" ;
	o8128:DM1619 "0" ;
	o8128:NumAdult "2" ;
	o8128:NumChild "0" ;
	o8128:NumSSex "0" ;
	o8128:NumCPart "0" ;
	o8128:NumMPart "2" ;
	o8128:NumCivP "0" ;
	o8128:DVHsize "2" ;
	o8128:Relsize "1" ;
	o8128:SelPer "1" ;
	o8128:CCPersNo "-1" ;
	o8128:Accom "1" ;
	o8128:Hhldr1 "1" ;
	o8128:Hhldr2 "1" ;
	o8128:Hhldr3 "0" ;
	o8128:Hhldr4 "0" ;
	o8128:Hhldr5 "0" ;
	o8128:Hhldr6 "0" ;
	o8128:Hhldr7 "0" ;
	o8128:Hhldr8 "0" ;
	o8128:Hhldr9 "0" ;
	o8128:Hhldr10 "0" ;
	o8128:HiHNum "1" ;
	o8128:Tenure "1" ;
	o8128:NumRooms "8" ;
	o8128:TVSet "1" ;
	o8128:TVSetNum "1" ;
	o8128:Cable "2" ;
	o8128:CableNum "-1" ;
	o8128:Games "2" ;
	o8128:GamesNum "-1" ;
	o8128:Land "1" ;
	o8128:LandNum "1" ;
	o8128:Mob "1" ;
	o

### Some SPARQL queries

### Which predicates are used?

This takes a minute to run

In [50]:
from pandas.io.json import json_normalize
import json
query="""
PREFIX o8128: <http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/>

SELECT DISTINCT ?p
WHERE
    {
        ?s ?p ?o .
    }
#LIMIT 100

"""
df=json_normalize(json.loads(g.query(query).serialize(format='json'))['results']['bindings'])
df['p.value']=df['p.value'].str.replace(r'http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/','o8128:')
df

,p.type,p.value
0,uri,o8128:Mob
1,uri,o8128:PaidTim6
2,uri,o8128:Relate6_P1
3,uri,o8128:Relate1_P1
4,uri,o8128:Help19
5,uri,o8128:NPaidT11
6,uri,o8128:NPaidDa3
7,uri,o8128:PaidHrs5
8,uri,o8128:Hhldr8
9,uri,o8128:Relate7_P2


### Number of adults in household

In [40]:

df

,o.type,o.value,p.type,p.value,s.type,s.value
0,literal,1,uri,o8128:Mob,bnode,N63bea323102f42be8b0509544f75e399
1,literal,-1,uri,o8128:PaidTim6,bnode,N7eaf5455902540068d1c796e22305daf
2,literal,-2,uri,o8128:Relate6_P1,bnode,N0b65372d14384d75857850d48322fa29
3,literal,0,uri,o8128:Relate1_P1,bnode,N0ebb075fb672496a9d9f7fd58d3addfe
4,literal,0,uri,o8128:Help19,bnode,N237febd49c2f4fd49564bd2e70002d2b
5,literal,-1,uri,o8128:NPaidT11,bnode,Nf3d9f65c36f6493ca259644982cdff55
6,literal,-1,uri,o8128:NPaidDa3,bnode,N789d5c23056641e792e1a6d9cae587f1
7,literal,-1,uri,o8128:PaidHrs5,bnode,N3f9956db66f54476bd2732006d96ba9a
8,literal,0,uri,o8128:Hhldr8,bnode,Nf27a3eaa51c0469a99fedf500ea6e4b6
9,literal,,uri,o8128:Relate7_P2,bnode,Nbf54717b93f1439eb5cc4924e89e28a1


In [44]:
df.dtypes

o.type     object
o.value    object
p.type     object
p.value    object
s.type     object
s.value    object
dtype: object

In [45]:
for x in g.query(query):
    print(x)

(rdflib.term.URIRef('http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/Mob'), rdflib.term.BNode('N63bea323102f42be8b0509544f75e399'), rdflib.term.Literal('1'))
(rdflib.term.URIRef('http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/PaidTim6'), rdflib.term.BNode('N7eaf5455902540068d1c796e22305daf'), rdflib.term.Literal('-1'))
(rdflib.term.URIRef('http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/Relate6_P1'), rdflib.term.BNode('N0b65372d14384d75857850d48322fa29'), rdflib.term.Literal('-2'))
(rdflib.term.URIRef('http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/Relate1_P1'), rdflib.term.BNode('N0ebb075fb672496a9d9f7fd58d3addfe'), rdflib.term.Literal('0'))
(rdflib.term.URIRef('http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/Help19'), rdflib.term.BNode('N237febd49c2f4fd49564bd2e70002d2b'), rdflib.term.Literal('0'))
(rdflib.term.URIRef('http://purl.org/berg/ontology/10.5255/UKDA-SN-8128-1/NPaidT11'), rdflib.term.BNode('Nf3d9f65c36f6493ca259644982cdff55'), rdflib.term.Literal